In [6]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [2]:
mpHolistic=mp.solutions.holistic
mpDraw=mp.solutions.drawing_utils

In [3]:
#draw landmarks in image
def drawLandmarks():
    mpDraw.draw_landmarks(img,results.face_landmarks,mpHolistic.FACEMESH_CONTOURS,mpDraw.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 256, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.pose_landmarks,mpHolistic.POSE_CONNECTIONS,mpDraw.DrawingSpec(color=(80, 22, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.left_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(121, 22, 90), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(255, 100, 112), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.right_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(250, 80, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(200, 50, 250), thickness=2, circle_radius=2))

In [4]:
# extract keypoints of the features in image
def extractKeypoints():
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)
    leftHand=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)
    rightHand=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()if results.right_hand_landmarks else np.zeros(21*3)
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([face,leftHand,rightHand,pose])

In [5]:
os.path.join("Sign_Images")

'Sign_Images'

In [4]:
#path for flatten datas
dataPath = os.path.join('Sign_Data')

#path for images
imgPath= os.path.join('Sign_Image')

signs=np.array(["Hello","Thank You", "Hungry", "Food", "Hospital", "Washroom"])

#30 video of each sign
numSequences=30

#length of each video
sequenceLength=30

In [7]:
for sign in signs:
    for sequence in range(numSequences):
            try:
                os.makedirs(os.path.join(dataPath,sign,str(sequence)))
                os.makedirs(os.path.join(imgPath,sign,str(sequence)))
            except:
                pass
# if not os.path.exists("Sign_Data"):
#     os.mkdir("Sign_Data")
# else:
#     print("already exist")


In [8]:
os.path??

In [9]:
# while True:
#     inpt=input(print("Select which data are you collecting:\n0: Hello \n1: Thank you"))
#     if inpt == '0' or inpt == '1':
#         choice=signs[int(inpt)]
#         break
#     else:
#         print("Invalid input, please select again!!")
# choice   

In [10]:
vid=cv2.VideoCapture(0)
with mpHolistic.Holistic() as holistic:

    #taking label input 
     while True:
        blankImg = np.zeros(shape=[512, 512, 3], dtype=np.uint8)
        cv2.putText(blankImg,"Select label: ", (10,50),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"0: Hello , 1: Thank You", (10,100),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"2: Hungry , 3: Food", (10,150),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"4: Hospital , 5: Washroom", (10,200),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"'ESC' to escape", (10,250),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.imshow('Select label',blankImg)
        inpt=cv2.waitKey(0)
        if inpt == 48 or inpt == 49 or inpt == 50 or inpt == 51 or inpt == 52 or inpt == 53:
            choice=signs[inpt-48]
            cv2.destroyWindow('Select label')
            break
        else:
            if inpt==27:
                break
            
    #checking camera is opened or not and taking data    
     while vid.isOpened() and inpt!=27 :
        for sequence in range(numSequences):
            for frameNum in range(sequenceLength+1):
                #checks for user input to close the windows                
                key=cv2.waitKey(1)                  
                success,img=vid.read()
                
                #checking if data is accessed or not from camera
                if not success:
                    break  
                img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                # img.flags.writeable=False
                results=holistic.process(img) 
                # img.flags.writeable=True     
                img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
                
                #draws landmarks
                drawLandmarks()
                
                #show feed for collecting datas and delays for 2 sec
                if frameNum == 0: 
                    cv2.putText(img, "Press 'ESC' to escape", (10,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) 
                    cv2.putText(img, 'press any key to conitnue', (10,60), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0, 0), 1, cv2.LINE_AA)
                    cv2.imshow('Collecting Datas', img)
                    key= cv2.waitKey(0)
                    cv2.waitKey(1000)
                    
                #starts collecting datas    
                else: 
                    cv2.putText(img, f"Collecting Data for '{choice}' Video Number {sequence}", (15,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) 
                    cv2.imshow('Collecting Datas', img)
                    keypoints=extractKeypoints()  
                    
                    #saving flatten array to datapath
                    np.save(os.path.join(dataPath,choice,str(sequence),str(frameNum-1)),keypoints)
                    
                    #saving image to image path
                    jpgPath=os.path.join(imgPath,choice,str(sequence),str(frameNum-1))
                    cv2.imwrite(f"{jpgPath}.jpg",img)
                    key=2 #giving default value for key to avoid esc while taking data
                
                if key == 27 : #press esc to close the window
                    break     

            if key == 27 : #press esc to close the window
                break        
        if key == 27 : #press esc to close the window
            break             
            
#releasing the port            
vid.release() 

#destroying all opened windows using opencv
cv2.destroyAllWindows()            

# Preprocess Data and Create Labels and Features

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
label_map = {label:num for num, label in enumerate(signs)}

In [13]:
label_map

{'Hello': 0,
 'Thank You': 1,
 'Hungry': 2,
 'Food': 3,
 'Hospital': 4,
 'Washroom': 5}

In [14]:
sequences, labels = [], []
for sign in signs:
    for sequence in range(numSequences):
        window = []
        for frameNum in range(sequenceLength):
            res = np.load(os.path.join(dataPath, sign, str(sequence), "{}.npy".format(frameNum)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[sign])

In [15]:
np.array(labels).shape

(180,)

In [16]:
np.array(sequences).shape

(180, 30, 1662)

In [17]:
arr=np.array([[[1,2],[3,4],[5,6]],[[1,2],[3,4],[5,6]]])

In [18]:
arr.shape

(2, 3, 2)

In [19]:
X = np.array(sequences)

In [20]:
X.shape

(180, 30, 1662)

In [21]:
y = to_categorical(labels).astype(int)

In [22]:
y.shape

(180, 6)

In [23]:
y

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [25]:
y_test.shape

(9, 6)

# Save test datas and signs

In [34]:
testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
try:
    os.makedirs(os.path.join(testDatasPath))
    os.makedirs(os.path.join(signsPath))
except:
    pass
np.save(os.path.join(testDatasPath,'x_test.npy'),X_test)
np.save(os.path.join(testDatasPath,'y_test.npy'),y_test)
np.save(os.path.join(signsPath,'signs.npy'),signs)

NameError: name 'X_test' is not defined

# Build and Train LSTM Neural Network

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [2]:
def createModel():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(signs.shape[0], activation='softmax'))
    return model

In [32]:
model=createModel()

In [29]:
res = [.7, 0.2, 0.1]

In [30]:
signs[np.argmax(res)]

'Hello'

In [31]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

NameError: name 'model' is not defined

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

# Make Prediction

In [20]:
#load signs, x_test, y_test
testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
x_test=np.load(os.path.join(testDatasPath,'x_test.npy'))
y_test=np.load(os.path.join(testDatasPath,'y_test.npy'))
signs=np.load(os.path.join(signsPath,'signs.npy'))

In [21]:
res = model.predict(x_test)

In [12]:
signs[np.argmax(res[5])]

'Hospital'

In [18]:
signs[np.argmax(y_test[5])]

'Hungry'

In [14]:
res.shape

(9, 6)

In [30]:
Y_test

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0]])

In [31]:
res

array([[0.15729538, 0.16477321, 0.15944463, 0.17102578, 0.17237754,
        0.17508347],
       [0.15855251, 0.1658583 , 0.16016524, 0.17001577, 0.17231709,
        0.17309111],
       [0.1611393 , 0.16691314, 0.1615812 , 0.16969882, 0.17082548,
        0.16984205],
       [0.15969467, 0.16563569, 0.16047707, 0.16989252, 0.17186616,
        0.17243387],
       [0.16217123, 0.17035839, 0.160731  , 0.16969222, 0.17100288,
        0.16604419],
       [0.1612128 , 0.1693305 , 0.16148058, 0.16964678, 0.1705475 ,
        0.16778187],
       [0.16161755, 0.17037238, 0.16123523, 0.16944562, 0.17053285,
        0.1667964 ],
       [0.16120632, 0.1671237 , 0.16145256, 0.1697366 , 0.17070429,
        0.1697766 ],
       [0.16313653, 0.16872379, 0.16225764, 0.16990489, 0.16965394,
        0.1663233 ]], dtype=float32)

In [32]:
np.argmax(res[0])

5

In [19]:
np.argmax(y_test[0])

0

# Save Model

In [ ]:
model.save('SLR.h5')

# Load Model

In [9]:
model=createModel()

In [10]:
model.load_weights('SLR.h5')

# Evaluation using confusion matrix

In [88]:
from sklearn.metrics import multilabel_confusion_matrix,confusion_matrix, accuracy_score

In [71]:
true_labels=[]
for label in y_test:
    index=np.argmax(label)
    true_labels.append(signs[index])

In [72]:
true_labels

['Hello',
 'Washroom',
 'Food',
 'Washroom',
 'Hungry',
 'Hungry',
 'Hungry',
 'Food',
 'Hospital']

In [104]:
y_predicted = model.predict(x_test)

In [73]:
predicted_labels=[]
for label in y_predicted:
    index=np.argmax(label)
    predicted_labels.append(signs[index])

In [74]:
predicted_labels

['Washroom',
 'Washroom',
 'Hospital',
 'Washroom',
 'Hospital',
 'Hospital',
 'Hospital',
 'Hospital',
 'Food']

In [101]:
y_true = np.argmax(y_test, axis=1).tolist()

In [105]:
y_predicted=np.argmax(y_predicted,axis=1).tolist()

In [91]:
y_true

[0, 0, 4, 2, 8, 1]

In [92]:
y_test

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0]])

In [93]:
y_test.shape

(9, 6)

In [106]:
y_predicted

[5, 5, 4, 5, 4, 4, 4, 4, 3]

In [110]:
signs

array(['Hello', 'Thank You', 'Hungry', 'Food', 'Hospital', 'Washroom'],
      dtype='<U9')

In [114]:
multilabel_confusion_matrix(true_labels, predicted_labels,labels=signs)

array([[[8, 0],
        [1, 0]],

       [[9, 0],
        [0, 0]],

       [[6, 0],
        [3, 0]],

       [[6, 1],
        [2, 0]],

       [[3, 5],
        [1, 0]],

       [[6, 1],
        [0, 2]]], dtype=int64)

In [108]:
confusion_matrix(y_true, y_predicted)

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 3, 0],
       [0, 0, 0, 2, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 2]], dtype=int64)

In [51]:
accuracy_score(y_true,y_predicted)

0.2222222222222222

In [60]:
y_test.tolist()

[[1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0]]

In [117]:
list(zip(true_labels, predicted_labels))

[('Hello', 'Washroom'),
 ('Washroom', 'Washroom'),
 ('Food', 'Hospital'),
 ('Washroom', 'Washroom'),
 ('Hungry', 'Hospital'),
 ('Hungry', 'Hospital'),
 ('Hungry', 'Hospital'),
 ('Food', 'Hospital'),
 ('Hospital', 'Food')]